# インベントリの動的生成

ここまでの演習で、Ansibleから操作する対象のサーバーはインベントリファイル（[02_server_ops](/edit/aitac-automation-handson/inventories/02_server_ops)）へ人がIPアドレス（やホスト名）を書き込んでいました。

もちろんこの方法でもAnsibleによる自動化は可能ですが、転記する際にミスしたり管理対象が多くなるとインベントリの管理自体も大変になってきます。

そこで、Ansible ではインベントリを動的に生成する手法が準備されています。この方法をDynamic Inventoryと呼びます。

## ダイナミックインベントリ

通常、Ansible から自動化を行う際は、

`ansible-playbook -i inventory_file_name playbook.yml`

という形式でコマンドを実行しますが、-i に続いて実行可能なファイル（スクリプトなど）を指定することで、そのスクリプトが返す値をインベントリとして利用することが可能です。

この「実行するとインベントリを生成するスクリプト」の要件は、「決められた形式（JSON）で値を返す」ということだけです。

[Dynamic Inventory](http://docs.ansible.com/ansible/latest/intro_dynamic_inventory.html)

スクリプトに使える言語には制限はありませんので、シェルスクリプト、PythonやRubyなど好きな言語で処理を記述できます。

ダイナミックインベントリが利用されるケースとして以下があります。

* 自動化の実行時にまだ対象が存在していないケース（今から実行する自動化で対象のサーバーが作られる）
* 大量のサーバーを管理する場合に、システムに問い合わせて接続先を取得するため、どれだけ対象が多くても人手によるミスが発生しない。

## ダイナミックインベントリのサンプル

スクリプトは一から作成することもできますが、デフォルトでいくつかのサンプルが付属しています。

[Dynamic Inventory Script](https://github.com/ansible/ansible/tree/devel/contrib/inventory)

上記のリンクを眺めると、EC2 や Azure、GCEといったメジャーなパブリッククラウドから、OpenStackやVMwareといったプライベートな基盤に対して問合せを行うスクリプトが準備されていることが確認できます。

ここでは、EC2用のサンプルを演習環境にダウンロードしてありますので、こちらを使ってみましょう。

In [1]:
# 実行例

./inventories/ec2.py --list

{
  "_meta": {
    "hostvars": {
      "13.112.122.31": {
        "ansible_host": "13.112.122.31", 
        "ec2__in_monitoring_element": false, 
        "ec2_account_id": "996856287150", 
        "ec2_ami_launch_index": "0", 
        "ec2_architecture": "x86_64", 
        "ec2_block_devices": {
          "sda1": "vol-083fbe3dd88212d8b"
        }, 
        "ec2_client_token": "151472089157447212", 
        "ec2_dns_name": "ec2-13-112-122-31.ap-northeast-1.compute.amazonaws.com", 
        "ec2_ebs_optimized": false, 
        "ec2_eventsSet": "", 
        "ec2_group_name": "", 
        "ec2_hypervisor": "xen", 
        "ec2_id": "i-0452e56226c443d76", 
        "ec2_image_id": "ami-4dd5522b", 
        "ec2_instance_profile": "", 
        "ec2_instance_type": "t2.micro", 
        "ec2_ip_address": "13.112.122.31", 
        "ec2_item": "", 
        "ec2_kernel": "", 
        "ec2_key_name": "aitac-automation-key", 
        "ec2_launch_time": "2017-12-31T11:48:15.000Z", 
        "ec2_monitor

In [ ]:
# この Cell を実行してください。1分ほどかかります。

./inventories/ec2.py --list

この ec2.py は先の演習で設定した AWS のアクセス情報を使って EC2 のAPIに問合せを行い、インスタンス一覧とそれぞれのインスタンスの情報を取得し、取得した情報を元にグループ仕分けした形式で返します。

上記の実行例では、

vpc_id vpc_16cb0c71 に接続しているインスタンスの一覧を、

```
"vpc_id_vpc_16cb0c71": [
  "13.112.122.31", 
  "13.115.202.69", 
  "13.230.56.76"
]
```

という形式で返しており、

インスタンスに付与された tag が automation_group=manager, automation_group=target を

```
  "tag_automation_group_manager": [
    "13.112.122.31"
  ], 
  "tag_automation_group_target": [
    "13.115.202.69", 
    "13.230.56.76"
  ], 
```

のように返しています。

つまり、この ec2.py を利用して以下のように実行すると、

In [ ]:
# この Cell を実行（1分ほどかかります）

ansible tag_automation_group_manager -i ./inventories/ec2.py \
        --private-key aitac-automation-key.pem \
        -u centos -b \
        -m ping

Tag で automation_group=manager が設定されたサーバーに対して処理を実行することができます。

さらに、以下のようにすると、

In [ ]:
# この Cell を実行（1分ほどかかります）

ansible tag_automation_group_target -i ./inventories/ec2.py \
        --private-key aitac-automation-key.pem \
        -u centos -b \
        -m ping

Tag で automation_group=target が設定されたサーバーに対して処理を実行できます。

## ダイナミックインベントリの利用

では実際に、Playbookと合わせてダイナミックインベントリを実行してみます。

ここで使うPlaybook は先の演習2で使ったものとほとんど同じですが、ターゲットとなるホストの指定方法が異なっています。

[server_ops_httpd_on_with_dinv.yml](/edit/aitac-automation-handson/server_ops_httpd_on_with_dinv.yml)

* 演習2では hosts: target （手動で作成したインベントリファイル内で定義したグループ）
* ここでは hosts: tag_automation_group_target （ダイナミックインベントリが返すグループ）

では、このPlaybookをダイナミックインベントリと合わせて実行してみましょう。

In [ ]:
# この Cell を実行（1分ほどかかります）

ansible-playbook -i ./inventories/ec2.py \
                 -u centos --private-key aitac-automation-key.pem \
                 server_ops_httpd_on_with_dinv.yml

実行結果は演習2と同じ状態になるはずです。

このケースではサーバーが2台しかないので、ダイナミックインベントリを利用してもさほどメリットを感じないかもしれません。しかし、これが50台のサーバーが対象の時はどうでしょうか？

または、インスタンスの起動 → サーバーの設定を連続して行いたい場合はどうでしょうか？

上記のようなケースでダイナミックインベントリは効果を発揮します。

本演習はここまでです。